In [151]:
import casadi
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML, display
from matplotlib.animation import FuncAnimation

In [175]:
m, l, g = 1, 1, 9.8

# Define optimization variables
T = 5.0  # Time horizon
N = 100  # Number of control intervals
dt = T / N

# define dynamics
nq = 1
nv = 1
cq = casadi.SX.sym("q", nq)  # generalized position
cv = casadi.SX.sym("v", nv)  # generalized velocity
cu = casadi.SX.sym("u", nv)  # control input
aba_fn = casadi.Function("aba_fn", [cq, cv, cu], [cu - g / l * casadi.sin(cq)])


# Euler integrator
def euler_integrate(q, v, u):
    q_next = q + v * dt
    v_next = v + aba_fn(q, v, u) * dt
    return q_next, v_next

In [ ]:
opti = casadi.Opti()

Q = opti.variable(nq, N + 1)
V = opti.variable(nv, N + 1)
U = opti.variable(nv, N)


# Set initial state
opti.subject_to(Q[:, 0] == np.array([0]))
opti.subject_to(V[:, 0] == np.array([0]))

# Set final state
opti.subject_to(Q[:, N] == np.array([np.pi]))
opti.subject_to(V[:, N] == np.array([0]))

# Set dynamics constraints
for k in range(N):
    q_next, v_next = euler_integrate(Q[:, k], V[:, k], U[:, k])
    opti.subject_to(Q[:, k + 1] == q_next)
    opti.subject_to(V[:, k + 1] == v_next)

# Set control limits
limit = 3.0
opti.subject_to(opti.bounded(-limit, U, limit))

# Define objective
obj = 0
for k in range(N):
    obj += U[:, k].T @ U[:, k]
    obj += (Q[:, k] - np.array([np.pi])) ** 2

opti.minimize(obj)

# Solve
opti.solver("ipopt")
sol = opti.solve()

In [ ]:
Q_traj = sol.value(Q)
V_traj = sol.value(V)

plt.plot(Q_traj, V_traj)
plt.xlabel("q")
plt.ylabel("v")
plt.grid()
plt.show()

In [ ]:
fig = plt.figure(figsize=(5, 5), constrained_layout=False)

ax1 = fig.add_subplot(111)
(ln1,) = ax1.plot([], [], linewidth=5, color="lightblue")
(ln2,) = ax1.plot([], [], marker=".", ls="", markersize=30)

# plt.axis('off')
plt.tight_layout()

ax1.set_xlim(-2, 2)
ax1.set_ylim(-2, 2)
ax1.set_aspect(1)


def update(i):
    q = Q_traj[i]
    x_start = [0, l * np.sin(q)]
    y_start = [0, -l * np.cos(q)]
    ln1.set_data(x_start, y_start)

    x_start = x_start[1:]
    y_start = y_start[1:]
    ln2.set_data(x_start, y_start)


anim = FuncAnimation(fig, update, np.arange(0, N - 1, 1), interval=dt * 1000)

print("Generating animation...")
plt.close()
html = HTML(anim.to_jshtml())
print("Done!")

In [ ]:
display(html)